In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

In [3]:
cohort = Cohort.load(id=1)

In [4]:
onset_dataframe = OnsetDataFrame.load(id=18)

In [17]:
feature_pipeline = FeaturePipeline.load(id=20)

In [18]:
import morpher
from morpher.jobs import *

In [19]:
regexes = [
    'any__Diagnosis__', 
    'any__Procedure__', 
    'any__Drug__', 
    'any__Material__', 
    'any__Encounter__', 
    'any__AlcoholUse__', 
    'any__DrugUse__', 
    'any__TobaccoUse__',
    'min__Height__', 
    'min__Weight__', 
    'min__VitalSign__', 
    'min__Measurement__', 
    'min__LabValue__', 
    'max__Height__', 
    'max__Weight__', 
    'max__VitalSign__', 
    'max__Measurement__', 
    'max__LabValue__'
]

agg_func_regex = ""
for r in regexes:
    agg_func_regex += r + "|"
agg_func_regex = agg_func_regex[:-1]

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]] = label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]].fillna(False)
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [20]:
from itertools import product

In [21]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 100, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -1, 30)]

In [22]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator if x[0] == x[2] and x[1] == x[3]]
targets = ['other_viral_infection_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [23]:
pipeline = training.execute(
    comment='baseline-3 ovi with RFE',
    version='33.0.0',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric",
    rfe__run=True,
    rfe__step_size=50
)

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f73f63ebcc0>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


other_viral_infection_onset_from_0_days_after_to_365_days_after
age_in_days
max__Height__HEIGHT
min__Height__HEIGHT
max__Weight__WEIGHT
min__Weight__WEIGHT
max__VitalSign__EPIC__(RESP)
max__VitalSign__EPIC__(T-AX)
max__VitalSign__EPIC__(T-O)
max__VitalSign__EPIC__(T-T)
max__VitalSign__EPIC__(T-TE)
max__VitalSign__EPIC__DBP
max__VitalSign__EPIC__PAIN SCORE
max__VitalSign__EPIC__PULSE
max__VitalSign__EPIC__PULSE OXIMETRY
max__VitalSign__EPIC__SBP
max__VitalSign__EPIC__TEMPERATURE
max__VitalSign__IBEX__(RESP)
max__VitalSign__IBEX__(T-AX)
max__VitalSign__IBEX__(T-O)
max__VitalSign__IBEX__(T-T)
max__VitalSign__IBEX__DBP
max__VitalSign__IBEX__PAIN SCORE
max__VitalSign__IBEX__PULSE
max__VitalSign__IBEX__PULSE OXIMETRY
max__VitalSign__IBEX__SBP
max__VitalSign__IBEX__TEMPERATURE
max__VitalSign__TDS__(P-A)
max__VitalSign__TDS__(P-R)
max__VitalSign__TDS__(RESP)
max__VitalSign__TDS__(T-AX)
max__VitalSign__TDS__(T-O)
max__VitalSign__TDS__(T-T)
max__VitalSign__TDS__ADMIT WT
max__VitalSign__TDS__APS 

Fitting estimator with 1174 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 164.708ms

Fitting estimator with 1124 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 884.841ms

Fitting estimator with 1074 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 154.725ms

Fitting estimator with 1024 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 732.44ms

Fitting estimator with 974 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 962.097ms

Fitting estimator with 924 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 685.373ms

Fitting estimator with 874 features.
*** Training of model 'G

...

Columns (11,18,19,20,24,27,35,64,83,90,92,94,95,96,98,105,112,113,115,118,122,123,130,133,137,138,139,140,141,147,168,169,175,232,241,245,246,285,308,313,314,316,321,325,329,336,348,354,355,356,360,376,391,396,414,418,423,427,429,432,439) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f73f4ad3c50>
Fetching data for Patient (...)
other_viral_infection_onset_from_0_days_after_to_365_days_after
age_in_days
any__Diagnosis__SYSTEM__MSDW_NOT APPLICABLE
any__Diagnosis__SYSTEM__MSDW_UNKNOWN
any__Procedure__SYSTEM__MSDW_NOT APPLICABLE
any__Material__SYSTEM__MSDW_NOT APPLICABLE
any__Encounter__Outpatient
gender_Female
gender_Male
gender_NOT AVAILABLE
race_African
race_Asian
race_Asian Chinese
race_Asian Indian
race_Asian Pacific
race_Black or African-American
race_Hispanic or Latino
race_Native American
race_Other
race_White


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 254.831ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 248.959ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 272.974ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 280.845ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 276.401ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 283.503ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 289.832ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 150.357ms

*** Training of model 'GradientBoostingClassifie

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
pipeline.id

202

# Results

In [27]:
from robotehr.api.training import get_training_results
get_training_results(202, sort_by='auc_roc_mean', metrics=['auc_roc_mean', 'precision_mean', 'auc_prc_mean', 'recall_mean'], n_rows=10000, response_type="pandas")

,algorithm,auc_prc_mean,auc_roc_mean,precision_mean,recall_mean,sampler,target,threshold_numeric,threshold_occurring,window_start_numeric,window_start_occurring
0,LogisticRegression,0.704575,0.810263,0.559345,0.675287,NoSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-331,-331
1,LogisticRegression,0.700965,0.809420,0.560054,0.659829,RandomOverSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-331,-331
2,LogisticRegression,0.704993,0.807498,0.544187,0.675258,SMOTE,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-331,-331
3,LogisticRegression,0.696937,0.803568,0.557936,0.649573,NoSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
4,LogisticRegression,0.697380,0.803313,0.554991,0.670203,SMOTE,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
5,LogisticRegression,0.694962,0.803001,0.561858,0.654686,RandomOverSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
6,LogisticRegression,0.686303,0.800974,0.523381,0.675258,RandomUnderSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-331,-331
7,LogisticRegression,0.684808,0.797971,0.549474,0.653050,NoSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-241,-241
8,LogisticRegression,0.666269,0.795642,0.537494,0.637533,NoSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-271,-271
9,LogisticRegression,0.682977,0.795468,0.539142,0.665001,RandomUnderSampler,other_viral_infection_onset_from_0_days_after_to_365_days_after,0.05,0.05,-361,-361
